__Описание__

В данном примере рассматривается простое удаление шума при помощи вейвлет анализа.

Данный пример может быть загружен по адресу: https://github.com/sven4500/num-analysis

In [ ]:
import numpy as np
import matplotlib
import pywt
import scipy
from matplotlib import pyplot

In [ ]:
def mse(a, b):
    return np.sqrt(np.sum(np.square(a - b)) / n)

In [ ]:
n = 2000
sigma = 0.1

t = np.linspace(0., 4. * np.pi, n, endpoint=False)
y = 2. * np.sin(t) + 0.5 * np.cos(3. * t)
y_noised = y + np.random.normal(0., sigma, size=n)

matplotlib.pyplot.plot(y_noised, color='orange')
matplotlib.pyplot.plot(y, color='black')

Очистка сигнала от шума заключается в 3 шагах:

1. Делаем прямое вейвлет преобразование;

2. Вычисляем порговое значение и оставляем только те дискреты которые больше порога;

3. Делаем обратное вейвлет преобразование.

Функция __wavedec__ производит прямое дискретное вейвлет преобразование (__анализ__) на заданном количестве уровней. Возвращает несколько массивов. Первый массив содержит аппроксимационные коэффициенты. Остальные массивы это детализирующие коэффициенты, каждый для своего уровня разложения.

In [ ]:
dwt = pywt.wavedec(y_noised, 'sym20', level=3)

for i in dwt[1:]:
    matplotlib.pyplot.plot(i)

Производим удаление ненужных коэффициентов по пороговому критерию.

In [ ]:
#dwt_approx_log2 = np.log2(np.abs(dwt_approx) + 1e-12)
#dwt_detail_log2 = np.log2(np.abs(dwt_detail) + 1e-12)

#th_approx = -np.sqrt(np.sum(np.square(dwt_approx_log2))) / len(dwt_approx_log2)
#th_detail = -np.sqrt(np.sum(np.square(dwt_detail_log2))) / len(dwt_detail_log2)

for i in dwt[1:]:
    n_level = len(i)
    sigma_level = np.sqrt(np.sum(np.square(i)) / n_level)
    th = sigma_level * np.sqrt(2 * np.log2(n_level) / n_level)
    i[:] = [j if np.abs(j) < th else 0. for j in i]

for i in dwt[1:]:
    matplotlib.pyplot.plot(i)

Делаем обратное вейвлет преобразование (__синтез__).

In [ ]:
y_denoised = pywt.waverec(dwt, wavelet='sym20')
print('MSE noise', mse(y, y_noised), 'MSE denoise', mse(y, y_denoised))

matplotlib.pyplot.plot(y_noised, 'orange')
matplotlib.pyplot.plot(y_denoised, 'black')

Делаем то же самое при помощи преобразования Фурье.

In [ ]:
fft = scipy.fft.fft(y_noised)
th = np.sqrt(np.sum(np.square(np.abs(fft))) / len(fft))
fft = [i if np.abs(i) > th else 0. for i in fft]
#matplotlib.pyplot.plot(np.abs(fft)[:50])

In [ ]:
y_denoised = scipy.fft.ifft(fft).real
print('MSE noise', mse(y, y_noised), 'MSE denoise', mse(y, y_denoised))

matplotlib.pyplot.plot(y_noised, 'orange')
matplotlib.pyplot.plot(y_denoised, 'black')

__Источники__

[1] Noise Reduction of Speech Signal using Wavelet Transform with Modified Universal Threshold // https://www.researchgate.net/publication/258493830_Noise_Reduction_of_Speech_Signal_using_Wavelet_Transform_with_Modified_Universal_Threshold

[2] Simple Denoising Algorithm Using Wavelet Transform // https://arxiv.org/abs/nlin/0002028

[3] Time Series Forecasting Using Wavelet Denoising an Application to Saudi Stock Index // https://www.sciencedirect.com/science/article/pii/S1018363918307554#bib12